## Imports

In [1]:
import pandas as pd
from datasets import Dataset

/opt/homebrew/Caskroom/miniforge/base/envs/ragas/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain_openai.chat_models import ChatOpenAI
from ragas.llms import LangchainLLMWrapper

llm_4o = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
llm_4o_mini = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))

/Users/shahules/ragas/src/ragas/prompt/base.py:9: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from ragas.llms.prompt import PromptValue


## Dataset

In [3]:
dataset = Dataset.from_json("datasets/dataset_v3.json")

In [4]:
result_df = pd.read_csv("results/default_prompt_demo.csv")
result_df = result_df[result_df["factual_correctness"]<1.0]

In [5]:
incorrect_indices = result_df.index.values.tolist()

In [6]:
dataset = dataset.select(incorrect_indices)

In [7]:
result_df

,user_input,response,reference,factual_correctness
0,How did the invention of the wheel impact anci...,# The Revolutionary Impact of the Wheel on Anc...,The invention of the wheel was a pivotal momen...,0.82
1,How did the discovery of fire impact early hum...,# The Transformative Power of Fire in Early Hu...,The discovery of fire was a pivotal moment in ...,0.99
6,What were the causes and impacts of The Black ...,# The Black Death: Causes and Impacts on Medie...,"The Black Death, which swept through Europe in...",0.93
7,What were the significant impacts of Columbus'...,# The Transformative Impact of Columbus' 1492 ...,Columbus' voyage to the Americas in 1492 had p...,0.98
8,What was the Protestant Reformation?,# The Protestant Reformation: A Revolution in ...,The Protestant Reformation was a 16th-century ...,0.98
9,What were the major impacts of the Industrial ...,# The Transformative Impacts of the Industrial...,"The Industrial Revolution, which began in the ...",0.91
11,What were the main causes and consequences of ...,# The French Revolution: Causes and Consequenc...,"The French Revolution, which began in 1789, wa...",0.99
12,How did the invention of the printing press im...,# The Printing Press: Catalyst of the Renaissa...,The invention of the printing press by Johanne...,0.96
13,What were the main factors that contributed to...,# The Fall of the Roman Empire: A Perfect Stor...,The fall of the Roman Empire was a complex pro...,0.98
17,What were the major causes and consequences of...,# The Major Causes and Consequences of World W...,World War II was primarily caused by the unres...,0.94


## Evaluation

In [8]:
import os
os.environ["LANGCHAIN_PROJECT"]= "Alignment"
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [9]:
from ragas.metrics import FactualCorrectness
from ragas import EvaluationDataset
from ragas import evaluate

In [10]:
ragas_dataset = EvaluationDataset.from_hf_dataset(dataset)

In [11]:
ragas_dataset

EvaluationDataset(features=['user_input', 'response', 'reference'], len=25)

In [12]:
ragas_output = evaluate(ragas_dataset[:10], metrics=[FactualCorrectness(with_examples=True)],raise_exceptions=True)

Evaluating: 100%|███████████████████████████████████████| 10/10 [00:38<00:00,  3.88s/it]


In [13]:
ragas_output.to_pandas()

,user_input,response,reference,factual_correctness
0,How did the invention of the wheel impact anci...,# The Revolutionary Impact of the Wheel on Anc...,The invention of the wheel was a pivotal momen...,0.83
1,How did the discovery of fire impact early hum...,# The Transformative Power of Fire in Early Hu...,The discovery of fire was a pivotal moment in ...,0.85
2,What were the causes and impacts of The Black ...,# The Black Death: Causes and Impacts on Medie...,"The Black Death, which swept through Europe in...",1.00
3,What were the significant impacts of Columbus'...,# The Transformative Impact of Columbus' 1492 ...,Columbus' voyage to the Americas in 1492 had p...,0.86
4,What was the Protestant Reformation?,# The Protestant Reformation: A Revolution in ...,The Protestant Reformation was a 16th-century ...,1.00
5,What were the major impacts of the Industrial ...,# The Transformative Impacts of the Industrial...,"The Industrial Revolution, which began in the ...",1.00
6,What were the main causes and consequences of ...,# The French Revolution: Causes and Consequenc...,"The French Revolution, which began in 1789, wa...",1.00
7,How did the invention of the printing press im...,# The Printing Press: Catalyst of the Renaissa...,The invention of the printing press by Johanne...,0.90
8,What were the main factors that contributed to...,# The Fall of the Roman Empire: A Perfect Stor...,The fall of the Roman Empire was a complex pro...,0.88
9,What were the major causes and consequences of...,# The Major Causes and Consequences of World W...,World War II was primarily caused by the unres...,0.94


## Prepare <Pos><Neg> Dataset

In [28]:
import pandas as pd
import json

In [31]:
df_neg = pd.read_csv("/Users/shahules/Downloads/nli_negative.csv")
df_pos = pd.read_csv("/Users/shahules/Downloads/nli_positive.csv")

In [34]:
annotated_samples = []
for neg_row in df_neg.itertuples():
    input_data = eval(neg_row.input_data)['context']
    for pos_row in df_pos.itertuples():
        if input_data == eval(pos_row.input_data)['context']:
            sample = {"input":input_data,
             "positive":pos_row.output_output,
             "negative":neg_row.output_output
            }
            annotated_samples.append(sample)

In [36]:
len(annotated_samples)

5

In [35]:
with open("results/nli_sample.json","w") as file:
    json.dump(annotated_samples,file,indent=4)